In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import glob

def data_generator(file_list, batch_size=4):
    """
    Generator that loads and yields batches on-the-fly
    Avoids loading all data into RAM
    """
    while True:
        # Shuffle files each epoch
        np.random.shuffle(file_list)
        
        for i in range(0, len(file_list), batch_size):
            batch_files = file_list[i:i+batch_size]
            images = []
            masks = []
            
            for file_path in batch_files:
                try:
                    with h5py.File(file_path, 'r') as f:
                        image = f['image'][:]
                        mask = f['mask'][:]
                        
                        # Only keep slices with substantial tumor
                        if np.sum(mask > 0) > 100:
                            # Normalize per-slice
                            image = (image - image.min()) / (image.max() - image.min() + 1e-8)
                            mask = (mask > 0).astype(np.float32)
                            
                            images.append(image)
                            masks.append(mask)
                except Exception as e:
                    print(f"Error loading {file_path}: {e}")
                    continue
            
            # Only yield if we have data
            if len(images) > 0:
                yield np.array(images), np.array(masks)

# Get all files
data_dir = './data/BraTS/BraTS2020_training_data/content/data'
all_files = glob.glob(f'{data_dir}/*.h5')
print(f"Total files found: {len(all_files)}")

# Shuffle and split 80/20
np.random.seed(42)
np.random.shuffle(all_files)
split_idx = int(0.8 * len(all_files))

train_files = all_files[:split_idx]
val_files = all_files[split_idx:]

print(f"Train files: {len(train_files)}")
print(f"Val files: {len(val_files)}")

# Create generators
train_gen = data_generator(train_files, batch_size=4)
val_gen = data_generator(val_files, batch_size=4)

Total files found: 57195
Train files: 45756
Val files: 11439


In [ ]:
# 1. Set seeds
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import layers, models

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# 2. Build model
def build_unet(input_shape=(240, 240, 4)):
    inputs = layers.Input(input_shape)
    
    # Encoder
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.Dropout(0.1)(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.Dropout(0.1)(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    
    # Bottleneck
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    c3 = layers.Dropout(0.2)(c3)
    
    # Decoder
    u4 = layers.UpSampling2D((2, 2))(c3)
    u4 = layers.concatenate([u4, c2])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    c4 = layers.Dropout(0.1)(c4)
    
    u5 = layers.UpSampling2D((2, 2))(c4)
    u5 = layers.concatenate([u5, c1])
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    c5 = layers.Dropout(0.1)(c5)
    
    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c5)
    
    return models.Model(inputs, outputs)

model = build_unet()

# 3. Define losses
def dice_coef(y_true, y_pred, smooth=1e-5):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def focal_loss(y_true, y_pred, alpha=0.25, gamma=2.0):
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
    bce = -(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    focal_weight = alpha * tf.pow(1 - p_t, gamma)
    return tf.reduce_mean(focal_weight * bce)

def combined_focal_dice(y_true, y_pred):
    return focal_loss(y_true, y_pred, alpha=0.25, gamma=0.2) + dice_loss(y_true, y_pred)

# 4. Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=combined_focal_dice,
    metrics=[dice_coef]
)

# 5. Callbacks
callbacks = [
    EarlyStopping(monitor='val_dice_coef', patience=20, mode='max', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, patience=5, mode='max', verbose=1),
    ModelCheckpoint('best_model.h5', monitor='val_dice_coef', mode='max', save_best_only=True)
]

# 6. Train with generators
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_files) // 4,
    validation_data=val_gen,
    validation_steps=len(val_files) // 4,
    epochs=150,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - dice_coef: 0.5520 - loss: 0.4573

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 3647s 319ms/step - dice_coef: 0.6364 - loss: 0.3687 - val_dice_coef: 0.7051 - val_loss: 0.2978 - learning_rate: 1.0000e-04
Epoch 2/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - dice_coef: 0.7063 - loss: 0.2965

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2864s 250ms/step - dice_coef: 0.7161 - loss: 0.2861 - val_dice_coef: 0.7197 - val_loss: 0.2810 - learning_rate: 1.0000e-04
Epoch 3/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - dice_coef: 0.7418 - loss: 0.2598

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2957s 259ms/step - dice_coef: 0.7435 - loss: 0.2579 - val_dice_coef: 0.7611 - val_loss: 0.2382 - learning_rate: 1.0000e-04
Epoch 4/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step - dice_coef: 0.7559 - loss: 0.2458

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 3571s 312ms/step - dice_coef: 0.7598 - loss: 0.2417 - val_dice_coef: 0.7629 - val_loss: 0.2350 - learning_rate: 1.0000e-04
Epoch 5/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - dice_coef: 0.7710 - loss: 0.2319

...

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2860s 250ms/step - dice_coef: 0.8742 - loss: 0.1268 - val_dice_coef: 0.8635 - val_loss: 0.1365 - learning_rate: 1.0000e-04
Epoch 39/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - dice_coef: 0.8756 - loss: 0.1257

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2848s 249ms/step - dice_coef: 0.8756 - loss: 0.1254 - val_dice_coef: 0.8653 - val_loss: 0.1344 - learning_rate: 1.0000e-04
Epoch 40/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - dice_coef: 0.8750 - loss: 0.1260

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2847s 249ms/step - dice_coef: 0.8757 - loss: 0.1252 - val_dice_coef: 0.8658 - val_loss: 0.1345 - learning_rate: 1.0000e-04
Epoch 41/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - dice_coef: 0.8769 - loss: 0.1241

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2843s 249ms/step - dice_coef: 0.8762 - loss: 0.1246 - val_dice_coef: 0.8667 - val_loss: 0.1336 - learning_rate: 1.0000e-04
Epoch 42/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2807s 245ms/step - dice_coef: 0.8758 - loss: 0.1246 - val_dice_coef: 0.8652 - val_loss: 0.1343 - learning_rate: 1.0000e-04
Epoch 43/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2940s 257ms/step - dice_coef: 0.8785 - loss: 0.1223 - val_dice_coef: 0.8551 - val_loss: 0.1447 - learning_rate: 1.0000e-04
Epoch 44/150
11439/11439 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - dice_coef: 0.8800 - loss: 0.1215

11439/11439 ━━━━━━━━━━━━━━━━━━━━ 2909s 254ms/step - dice_coef: 0.8792 - loss: 0.1219 - val_dice_coef: 0.8694 - val_loss: 0.1306 - learning_rate: 1.0000e-04
Epoch 45/150
  101/11439 ━━━━━━━━━━━━━━━━━━━━ 43:17 229ms/step - dice_coef: 0.8859 - loss: 0.1171

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.subplot(1, 2, 2)
plt.plot(history.history['dice_coef'], label='Train Dice')
plt.plot(history.history['val_dice_coef'], label='Val Dice')
plt.legend()
plt.title('Dice Coefficient')
plt.show()

In [ ]:
# Predict on validation set
predictions = model.predict(X_val)

# Visualize
fig, axes = plt.subplots(3, 3, figsize=(12, 12))

for i in range(3):
    axes[i, 0].imshow(X_val[i, :, :, 0], cmap='gray')
    axes[i, 0].set_title('MRI Input')
    
    axes[i, 1].imshow(y_val[i, :, :, 0], cmap='Reds')
    axes[i, 1].set_title('Ground Truth')
    
    axes[i, 2].imshow(predictions[i, :, :, 0] > 0.5, cmap='Reds')
    axes[i, 2].set_title('Prediction')

plt.tight_layout()
plt.show()